####Socket Programming
<img src='tcp_connection_sequence.png'>

```python
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
```

```python
# Bind the socket to the port
server_address = ('localhost', 9876)
print >>sys.stderr, 'starting up on %s port %s' % server_address
socket.bind(server_address)
```

Calling listen() puts the socket into server mode, and accept() waits for an incoming connection.

```python
# Listen for incoming connections
sock.listen(1)

while True:
    # Wait for a connection
    print >>sys.stderr, 'waiting for a connection'
    connection, client_address = sock.accept()
```

* accept() returns an open connection between the server and client, along with the address of the client. 
* The connection is actually a different socket on another port (assigned by the kernel). 
* Data is read from the connection with recv() and transmitted with sendall().

```python
    try:
        print >>sys.stderr, 'connection from', client_address

        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(16)
            print >>sys.stderr, 'received "%s"' % data
            if data:
                print >>sys.stderr, 'sending data back to the client' 
                connection.sendall(data)
            else:
                print >>sys.stderr, 'no more data from', client_address
                break
            
    finally:
        # Clean up the connection
        connection.close()
```

###Connecting a client
```python
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('localhost', 9876)
print >>sys.stderr, 'connecting to %s port %s' % server_address
sock.connect(server_address)
```

###communicating with the server
```python
try:
    
    # Send data
    message = 'This is the message.  It will be repeated.'
    print >>sys.stderr, 'sending "%s"' % message
    sock.sendall(message)

    # Look for the response
    amount_received = 0
    amount_expected = len(message)
    
    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print >>sys.stderr, 'received "%s"' % data

finally:
    print >>sys.stderr, 'closing socket'
    sock.close()
```

### Extending Python:


#### c program
```c
/* File: example.c */

#include "example.h"

int fact(int n) {
    if (n < 0){ /* This should probably return an error, but this is simpler */
        return 0;
    }
    if (n == 0) {
        return 1;
    }
    else {
        /* testing for overflow would be a good idea here */
        return n * fact(n-1);
    }
}
```

#### header file

```c
/* File: example.h */

int fact(int n);
```

#### SWIG wrapper

```c
/* File: example.i */
%module example

%{
#define SWIG_FILE_WITH_INIT
#include "example.h"
%}

int fact(int n);
```

 The #define SWIG_FILE_WITH_INIT line inserts a macro that specifies that the resulting C file should be built as a python extension, inserting the module init code.

To build a Python module, run SWIG using the -python option:
```shell
swig -python example.i
```

This creates example_wrap.c (or example_wrap.cxx if -c++ option was given)  and a Python source file example.py

The setup call then sets up distutils to build your package

```shell
python setup.py build_ext --inplace
```


- python -- the version of python you want to build for
- setup.py -- the name of your setup script (it can be called anything, but setup.py is the tradition)
- build_ext -- telling distutils to build extensions
- --inplace -- this tells distutils to put the extension lib in the current dir. 

### Now use inside python!
```python
from example import fact
fact(10)
```

### Other options:
http://intermediate-and-advanced-software-carpentry.readthedocs.org/en/latest/c++-wrapping.html

- Manual wrapping (PyObject) 
- Wrapping C code with pyrex
- ctypes
- SIP (Python bindings for Qt)
- Boost.Python

